# Air Temperatures

Below are a series of graphs showing air temperatures at various sites in the watersheds. 

In [2]:
# Import libraries
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
import altair_saver 
import seaborn as sns

In [3]:
os.chdir('C://Users/pmarshal/Documents/Climate-Outlook/monthly-climate')
os.getcwd()

'C:\\Users\\pmarshal\\Documents\\Climate-Outlook\\monthly-climate'

In [4]:
# Import dataset
orc_temp = pd.read_csv('data/OrchidTA2.csv', parse_dates=['date']) 
orc_temp['year'] = pd.DatetimeIndex(orc_temp['date']).year
orc_temp['month'] = pd.DatetimeIndex(orc_temp['date']).month
orc_temp['day'] = pd.DatetimeIndex(orc_temp['date']).day
orc_temp['DOY'] = pd.DatetimeIndex(orc_temp['date']).dayofyear
orc_temp.head()

,date,mean_TA,year,month,day,DOY
0,2001-01-01,1.3,2001,1,1,1
1,2001-01-02,5.5,2001,1,2,2
2,2001-01-03,2.6,2001,1,3,3
3,2001-01-04,1.6,2001,1,4,4
4,2001-01-05,0.5,2001,1,5,5


In [5]:
# Claculate statistics on mean temp column
orc_temp['mean'] = orc_temp.groupby('DOY')['mean_TA'].transform('mean')
orc_temp['max'] = orc_temp.groupby('DOY')['mean_TA'].transform('max')
orc_temp['min'] = orc_temp.groupby('DOY')['mean_TA'].transform('min')
orc_temp['std'] = orc_temp.groupby('DOY')['mean_TA'].transform('std')
orc_temp['sem'] = orc_temp.groupby('DOY')['mean_TA'].transform('sem')
orc_temp['ci95_hi'] = orc_temp['mean'] + 1.96* orc_temp['sem']
orc_temp['ci95_lo'] = orc_temp['mean'] - 1.96* orc_temp['sem']


In [6]:
# Create dataframe with data for the current year
orc_current_year = orc_temp.loc[orc_temp['year'] == 2023]

In [7]:
# Create dataframe with historical data (i.e. not == current year)
orc_past = orc_temp.loc[orc_temp['year'] != 2023]
#orc_past = orc_past.dropna()
orc_past_stats = orc_past.loc[0:364]

In [8]:
# Calculate past low temperatures
orc_pastlow = orc_past.groupby('DOY')['mean_TA'].transform('min')
orc_pastlow = orc_pastlow.reset_index()
orc_pastlow = orc_pastlow.loc[0:364]
orc_pastlow['DOY'] = orc_past['DOY']
orc_pastlow = orc_pastlow.rename(columns={"mean_TA": "low"})
# Calculate past high temperatures
orc_pasthigh = orc_past.groupby('DOY')['mean_TA'].transform('max')
orc_pasthigh = orc_pasthigh.reset_index()
orc_pasthigh = orc_pasthigh.loc[0:364]
orc_pasthigh['DOY'] = orc_past['DOY']
orc_pasthigh = orc_pasthigh.rename(columns={"mean_TA": "high"})

In [9]:
# Calculate new lows this year
orc_present_low = pd.merge(orc_current_year, orc_pastlow)
orc_present_low = orc_present_low[orc_present_low['mean_TA'] <= orc_present_low['low']]
# Calculate new highs this year
orc_present_high = pd.merge(orc_current_year, orc_pasthigh)
orc_present_high = orc_present_high[orc_present_high['mean_TA'] >= orc_present_high['high']]


In [10]:
orc_plot_2023 = alt.Chart(orc_current_year).mark_line(color='darkred', strokeWidth=1.5).encode(
    alt.X('monthdate(date)'),
    alt.Y('mean_TA'),
    tooltip=[alt.Tooltip('monthdate(date):T', title="Date"), alt.Tooltip('mean_TA:Q', title="Air Temp")]
)

orc_plot_lows = alt.Chart(orc_present_low).mark_circle(color='blue', size=40).encode(
    alt.X('monthdate(date)'),
    alt.Y('mean_TA')
)

orc_plot_highs = alt.Chart(orc_present_high).mark_circle(color='red', size=40).encode(
    alt.X('monthdate(date)'),
    alt.Y('mean_TA')
)

orc_plot_current = orc_plot_2023 + orc_plot_lows + orc_plot_highs


In [11]:
title1 = alt.TitleParams(
   text='Orchid Lake - Seymour Watershed (1200m)',
   subtitle="Mean Daily Temperature (2001-2023)",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

orc_area = alt.Chart(orc_past_stats, title=title1).mark_area(color='#7DA6F3', opacity=0.5).encode(
    alt.X('monthdate(date)', title=' '),
    alt.Y('max:Q', title='Air Temperature (C)'),
    alt.Y2('min:Q')
).properties(width=600, height=300)

orc_area2 = alt.Chart(orc_past_stats).mark_area(color='#919397', opacity=0.6).encode(
    alt.X('monthdate(date)'),
    alt.Y('ci95_hi:Q'),
    alt.Y2('ci95_lo:Q')
)


orc_area + orc_area2  + orc_plot_current.interactive()

alt.LayerChart(...)

In [38]:
# Import dataset
#orc_monthly = orc_temp.groupby('month')['mean_TA'].mean().reset_index()
orc_monthly = orc_temp.groupby(pd.PeriodIndex(orc_temp['date'], freq="M"))['mean_TA'].mean().reset_index()
orc_monthly['date'] = pd.to_datetime(orc_monthly['date'].astype(str) + '-01')
orc_monthly['year'] = pd.DatetimeIndex(orc_monthly['date']).year
orc_monthly['month'] = pd.DatetimeIndex(orc_monthly['date']).month
orc_monthly.head()

,date,mean_TA,year,month
0,2001-01-01,0.429032,2001,1
1,2001-02-01,-1.450000,2001,2
2,2001-03-01,0.819355,2001,3
3,2001-04-01,2.083333,2001,4
4,2001-05-01,6.719355,2001,5


In [39]:
# Calculate the average monthly temperature (mean TA)
orc_average_monthly_ta = orc_monthly.groupby('month')['mean_TA'].mean().reset_index()
orc_average_monthly_ta.rename(columns={'mean_TA': 'TA'}, inplace=True)
# Merge the average monthly temperature values with the 'monthly' DataFrame
orc_monthly = orc_monthly.merge(orc_average_monthly_ta, on='month', how='left')
orc_monthly.head()

,date,mean_TA,year,month,TA
0,2001-01-01,0.429032,2001,1,-0.219208
1,2001-02-01,-1.450000,2001,2,-1.146104
2,2001-03-01,0.819355,2001,3,0.210850
3,2001-04-01,2.083333,2001,4,3.125758
4,2001-05-01,6.719355,2001,5,7.817889


In [40]:
# Calculate the temperature anomaly (TA - Mean_TA)
orc_monthly['TA_Anomaly'] = orc_monthly['mean_TA'] - orc_monthly['TA']

# Format the 'TA' and 'TA_Anomaly' columns to have 2 decimal places
orc_monthly['TA'] = orc_monthly['TA'].round(2)
orc_monthly['mean_TA'] = orc_monthly['mean_TA'].round(2)
orc_monthly['TA_Anomaly'] = orc_monthly['TA_Anomaly'].round(2)
orc_monthly.head()

,date,mean_TA,year,month,TA,TA_Anomaly
0,2001-01-01,0.43,2001,1,-0.22,0.65
1,2001-02-01,-1.45,2001,2,-1.15,-0.30
2,2001-03-01,0.82,2001,3,0.21,0.61
3,2001-04-01,2.08,2001,4,3.13,-1.04
4,2001-05-01,6.72,2001,5,7.82,-1.10


## Monthly Average Air Temperatures
The boxplot below shows average monthly air temperatures for the current year (blue or red ticks), along with the median, first and third quartile, and the range of max and min. Hover over the plot for values. 

In [45]:
title2 = alt.TitleParams(
   text='Orchid Lake - Seymour Watershed (1200m)',
   subtitle="Monthly Temperatures (2001-2023)",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

# Filter the data for the current year (2023)
orc_data_2023 = orc_monthly[orc_monthly['year'] == 2023]

# Create the boxplot for historical precipitation
orc_boxplot_TA = alt.Chart(orc_monthly, title=title2).mark_boxplot(extent='min-max', opacity=0.5).encode(
    alt.X('month:O', title='Month (January-December)'),  # Display month in "mmm" format
    alt.Y('mean_TA:Q', title='Air Temperature (C)'),
).properties(width=450, height=200)

#Create a base chart for 'Mean_TA' values
orc_mean_ta_line = alt.Chart(orc_data_2023).mark_tick(size=12, thickness=2.5).encode(
    x=alt.X('month:O', title='Month (January-December)'),
    y=alt.Y('mean_TA:Q', title='Air Temperature (C)', scale=alt.Scale(domain=[-6, 22])),
    color=alt.condition(
        alt.datum.TA_Anomaly > 0,
        alt.value("red"),  # The positive color
        alt.value("blue")  # The negative color
    ),
    tooltip=[alt.Tooltip('date', title='Date'), alt.Tooltip('mean_TA', title='Temp')]
).properties(
    width=450, height=200
)

# Create a chart for 'TA_Anomaly'
orc_anomaly_chart = alt.Chart(orc_data_2023).mark_bar().encode(
    x=alt.X('TA_Anomaly:Q', title=None),
    y=alt.Y('month:O', title='Month'),
    color=alt.condition(
        alt.datum.TA_Anomaly > 0,
        alt.value("red"),  # The positive color
        alt.value("blue")  # The negative color
    ),
    tooltip=['TA_Anomaly:Q', 'date:T']
).properties(
    title = 'Temp Anomaly (C)',width=100, height=200
)


# Combine the two charts
orc_combined_TA = orc_boxplot_TA + orc_mean_ta_line
orc_final_chart = orc_combined_TA | orc_anomaly_chart 

orc_final_chart

alt.HConcatChart(...)

In [18]:
qd_stripes_df = qd_monthly.drop(['Precip', 'Mean', 'percent_norm', 'Mean_TA', 'TA_Anomaly'], axis=1)

In [85]:
annual_avg_temps = qd_stripes_df.groupby('year')['TA'].mean().reset_index().round(2)
annual_avg_temps = annual_avg_temps.iloc[0:20]
# Calculate the average monthly temperature (mean TA)
annual_temps = annual_avg_temps['TA'].mean()

In [86]:
annual_avg_temps['moving_avg'] = annual_avg_temps['TA'].rolling(5).mean()

## Annual Air Temperature
The plot below shows the average annual air temperatures in the water supply areas. In general, the length of record at these stations is not long enough to show warming from climate change. Variability is linked more to climate drivers like the El Nino - Southern Oscillation (ENSO). 

In [89]:
title3 = alt.TitleParams(
   text='Lower Capilano Watershed (250m)',
   subtitle="Mean annual air temperatures with running 5-year mean",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

bar_chart = alt.Chart(annual_avg_temps, title=title3).mark_bar(opacity=0.8).encode(
    alt.X('year:O', title=None),
    alt.Y('TA:Q', title='Air Temperature (C)', scale=alt.Scale(domain=[0, 12])),
    color=alt.condition(
        alt.datum.TA > '9.07',
        alt.value("red"),  # The positive color
        alt.value("steelblue")  # The negative color
    ),
    tooltip=[alt.Tooltip('year', title='Year'), alt.Tooltip('TA', title='Temp')]
).properties(
    width=600,
    height=200
)

line = alt.Chart(annual_avg_temps).mark_line(strokeDash=[4, 2], color='black', opacity=0.6).encode(
    alt.X('year:O'),
    alt.Y('moving_avg:Q')
)

bar_chart + line

alt.LayerChart(...)